In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2016.json"

response = requests.get(URL)
with open("index_2016.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2016.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
378415,742320663,201509,93493228037496,990,https://s3.amazonaws.com/irs-form-990/20164228...,CASA DE AMIGOS OF WILLACY COUNTY INC,2016-12-29,201642289349303749,2017-01-11T22:15:19.7260064Z
378416,581897269,201509,93493228037946,990,https://s3.amazonaws.com/irs-form-990/20164228...,ANNIE PENN MEMORIAL HOSPITAL FOUNDATION,2016-12-29,201642289349303794,2017-01-11T22:15:19.7260064Z
378417,566049500,201512,93493228038096,990,https://s3.amazonaws.com/irs-form-990/20164228...,NC MASONIC FOUNDATION INC,2016-12-29,201642289349303809,2017-01-11T22:15:19.7260064Z
378418,592374971,201603,93493228038146,990,https://s3.amazonaws.com/irs-form-990/20164228...,SOUTH HILLSBOROUGH LODGE NO 2672 BPOE OF THE USA,2016-12-29,201642289349303814,2017-01-11T22:15:19.7260064Z
378419,237107846,201512,93492135010746,990EZ,https://s3.amazonaws.com/irs-form-990/20164135...,SOUTH TOMS RIVER VOLUNTEER FIRST AID SQUAD INC,2016-12-28,201641359349201074,2017-01-11T22:15:19.7280064Z


In [5]:
df.shape

(378420, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2016/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [7]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2016/'):
    f.extend(filenames)
    break

In [8]:
u = list(df['URL'])
len(u)

378420

In [9]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [10]:
print(len(file_names_list), str(":"), len(f))

378420 : 378177


In [11]:
diff = list(set(file_names_list) - set(f))
len(diff)

243

In [12]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [13]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [14]:
for url in urls:
    download_url(url)

In [15]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2016/'):
    f.extend(filenames)
    break

In [16]:
len(f)

378420

In [17]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [18]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [19]:
print(list_duplicates(file_names_list))

[]


In [20]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [21]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [22]:
dup_ein

,EIN


In [23]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [24]:
duplicates

""
